Anexo S

# Desplegando modelo EfficientNet en IBM Watson

#### Elaborado por: Ricardo Niño de Rivera Barrón

#### Ingeniería Biónica

#### Trabajo Terminal II

En esta libreta interactiva realizada en IBM Watson se describe el proceso para desplegar un modelo EfficientNet en la nube utilizando las herramientas que proporciona gratuitamente IBM.

## Instalando e Importando biblioteca EfficientNet

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
!conda install efficientnet

In [ ]:
# Importando todos los métodos
# from efficientnet.keras import EfficientNetB7
import efficientnet.tfkeras as efn

## Configurando el ambiente

Primeramente es necesario crear una instancia gratuita en Watson para utilizar el servicio específico de Machine Learning.

### Conectando a Watson Machine Learning

Para comunicarnos con la instancia creada necesitamos dos datos, "Api Key" y la ubicación geográfica de la instancia.


In [1]:
api_key = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
location = 'us-south'

In [2]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Importando la biblioteca para manipular recursos en IBM Watson

In [3]:
#!pip install -U ibm-watson-machine-learning

In [4]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

2020-12-04 02:02:55,297 - tensorflow - WARNING - Large dropout rate: 0.5125 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
2020-12-04 02:02:55,516 - tensorflow - WARNING - Large dropout rate: 0.525 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
2020-12-04 02:02:55,750 - tensorflow - WARNING - Large dropout rate: 0.5375 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
2020-12-04 02:02:55,983 - tensorflow - WARNING - Large dropout rate: 0.55 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
2020-12-04 02:02:56,233 - tensorflow - WARNING - Large dropout rate: 0.5625 (>0.5). In TensorFlow 2.x, dropout() uses dropout rate instead of keep_prob. Please ensure that this is intended.
2020-12-04 02:09:10,736 - ibm_watson_machine_learning

### Trabajando con espacios

Posteriormente se debe establecer un "espacio" de trabajo en en en el servicio de almacenamiento de objetos de IBM Cloud.

Para realizar esta tarea es posible realizarla en esta libreta o directamente en la plataforma.

In [5]:
space_id = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'

Enlistando todos los espacios existentes en la instancia creada.

In [6]:
client.spaces.list(limit=10)

------------------------------------  ----------------  ------------------------
ID                                    NAME              CREATED
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNet_TT2  2020-12-03T14:58:40.571Z
------------------------------------  ----------------  ------------------------


Ahora establecemos el espacio creado como espacio por default para desplegar el modelo propuesto.

In [7]:
client.set.default_space(space_id)

'SUCCESS'

In [8]:
client.deployments.get_job_details()

{'resources': []}

In [9]:
!ls

## Descargando ejemplo de prueba

In [10]:
# Instalando biblioteca mega.py
!pip install mega.py

     |████████████████████████████████| 49 kB 7.4 MB/s  eta 0:00:01
     |████████████████████████████████| 13.7 MB 20.4 MB/s eta 0:00:01
  Created wheel for pathlib: filename=pathlib-1.0.1-py3-none-any.whl size=14346 sha256=d1816d7868fc64c2fc0c203e9f7c2ed34ee943448337fdba354598172f9ad9da
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/6e/96/b8/10037fe231e23970bac58361d7c93571ab983a7bbc55e68550
Successfully built pathlib


In [11]:
from mega import Mega

In [12]:
mega = Mega()

Inicamos sesión en MEGA con cuenta temporal y anónima

In [13]:
# Log in en la cuenta de MEGA con cuenta temporal anónima
m = mega.login()

In [14]:
# Descargando el archivo de prueba T0032.1.1.D.2016-10-29.16.npy
m.download_url('https://mega.nz/file/URwlCCpC#It8L-P9xJexCtwgFY2q1NbXkEH8A9aVg9JWZfedLLjw')

PosixPath('T0032.1.1.D.2016-10-29.16.npy')

In [15]:
# Mostrando archivos en el directorio de trabajo del entorno
!ls

T0032.1.1.D.2016-10-29.16.npy


In [16]:
# Imprimiendo el directorio de trabajo
!pwd

/home/wsuser/work


Leyendo archivo con numpy.

In [17]:
import numpy as np

In [18]:
X_prueba = np.load('T0032.1.1.D.2016-10-29.16.npy')
X_prueba.shape

(480, 640)

## Acondicionando imagen

In [19]:
# # importando bibliotecas auxiliares
import tensorflow as tf
# from tensorflow.keras.applications import *
from tensorflow.keras.layers import *

In [20]:
# Definiendo función get_img para leer los archivos numpy y ajustarlos a nuevo tamaño.
# La imagen que retorna la función también es normalizada sobre los valor máximo del pixel (255)
def get_img(imagen, size=128):
    
    img = np.expand_dims(imagen, axis=2)
    
    return tf.image.resize(img, [size,size], antialias=True)/255

In [21]:
X_prueba = get_img(X_prueba)
X_prueba.shape

TensorShape([128, 128, 1])

In [22]:
# X_prueba_128

X_prueba_128 = np.zeros((1, 128, 128,3))

# Almacenamos la imagen obtenida en cada canal
X_prueba_128[0,:,:,0] = X_prueba[:,:,0]
X_prueba_128[0,:,:,1] = X_prueba[:,:,0]
X_prueba_128[0,:,:,2] = X_prueba[:,:,0]

In [23]:
!ls

T0032.1.1.D.2016-10-29.16.npy


### Descargando los archivos del modelo EfficientNet 

Descargando archivos desde Kaggle.

In [24]:
# Descargando archivo con modelo EficienNetB7 EFNB7_weights.h5
!wget https://www.kaggleusercontent.com/kf/48361538/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..06envnQufMII0qsVqfsHRA.7zF6cLYBvn-y2o6LCup5VFIuTqQJTyoP8ahEkiUOe88VLrnCcuKRGnJl5jvA8BIBQoLGo2YSh_Pnv8qGvaiss7p0The2f3wfN286A-3riDBIDMRbzvNY_9XpeuigFExcgm199G69Z0qegjGEP_QknHKrDfZvb3ln5XyLBdrkvL2fvpfOPLFviItY1SR6qnCuUDpQy3qNTVoKFlHBf2ajwoo8Ss_Aaev2CRRb0tDpKPJq52Cbh5J5I4lwa9xvHrnDH5gqZW3dgz3PIx-E7kr6SrEhqEcNyCVTgGJXcpI8TYVDo7HZ48jZxJDuNLFf64AJkEmux9pgJURi0UBKiWA2mUuQ-ykWHcYErtkflAZjurB-om42CxTmBMOUZh1k97G6-Cv-7aWhE3PrOC77teE5OCPXEfeHpWicEQ4vCV3ow_RMwM-UxlIx4nfbkCmi9x8Qg7lKNtDv1HZ3-rsrofG0E9YvaKBhoQMbQpoJrGY6UHZ8iOdngnHUF1ogImA8Fk4kGceXRRFf0OWnEwgtH7bKz2ohwtsZ2T6QJHdgxWO_kfox9mUOr3wx-8ijD_hkvu0CqNLvZMoZF1awIycTZ6A0FCxwPgZj3Kb7ZD0uPRU4Cojpac22R92Vxq4U2IbHv7lg.g8Py8ocuWzzMZfiOCkYN0g/EFNB7_weights.h5

--2020-12-04 02:02:03--  https://www.kaggleusercontent.com/kf/48361538/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..06envnQufMII0qsVqfsHRA.7zF6cLYBvn-y2o6LCup5VFIuTqQJTyoP8ahEkiUOe88VLrnCcuKRGnJl5jvA8BIBQoLGo2YSh_Pnv8qGvaiss7p0The2f3wfN286A-3riDBIDMRbzvNY_9XpeuigFExcgm199G69Z0qegjGEP_QknHKrDfZvb3ln5XyLBdrkvL2fvpfOPLFviItY1SR6qnCuUDpQy3qNTVoKFlHBf2ajwoo8Ss_Aaev2CRRb0tDpKPJq52Cbh5J5I4lwa9xvHrnDH5gqZW3dgz3PIx-E7kr6SrEhqEcNyCVTgGJXcpI8TYVDo7HZ48jZxJDuNLFf64AJkEmux9pgJURi0UBKiWA2mUuQ-ykWHcYErtkflAZjurB-om42CxTmBMOUZh1k97G6-Cv-7aWhE3PrOC77teE5OCPXEfeHpWicEQ4vCV3ow_RMwM-UxlIx4nfbkCmi9x8Qg7lKNtDv1HZ3-rsrofG0E9YvaKBhoQMbQpoJrGY6UHZ8iOdngnHUF1ogImA8Fk4kGceXRRFf0OWnEwgtH7bKz2ohwtsZ2T6QJHdgxWO_kfox9mUOr3wx-8ijD_hkvu0CqNLvZMoZF1awIycTZ6A0FCxwPgZj3Kb7ZD0uPRU4Cojpac22R92Vxq4U2IbHv7lg.g8Py8ocuWzzMZfiOCkYN0g/EFNB7_weights.h5
Resolving www.kaggleusercontent.com (www.kaggleusercontent.com)... 35.190.26.106
Connecting to www.kaggleusercontent.com (www.kaggleusercontent.com)|35.190.26.106|:443... conne

In [25]:
!ls

EFNB7_weights.h5  T0032.1.1.D.2016-10-29.16.npy


## Definiendo EfficientNet en https://github.com/qubvel/efficientnet/blob/master/efficientnet/model.py

Importando bibliotecas y métodos de tensorflow

In [26]:
from tensorflow import keras

In [27]:
from tensorflow.keras.models import load_model

In [28]:
tf.__version__

'2.1.0'

In [29]:
def _obtain_input_shape(input_shape,
                        default_size,
                        min_size,
                        data_format,
                        require_flatten,
                        weights=None):
    """Internal utility to compute/validate a model's input shape.
    # Arguments
        input_shape: Either None (will return the default network input shape),
            or a user-provided shape to be validated.
        default_size: Default input width/height for the model.
        min_size: Minimum input width/height accepted by the model.
        data_format: Image data format to use.
        require_flatten: Whether the model is expected to
            be linked to a classifier via a Flatten layer.
        weights: One of `None` (random initialization)
            or 'imagenet' (pre-training on ImageNet).
            If weights='imagenet' input channels must be equal to 3.
    # Returns
        An integer shape tuple (may include None entries).
    # Raises
        ValueError: In case of invalid argument values.
    """
    if weights != 'imagenet' and input_shape and len(input_shape) == 3:
        if data_format == 'channels_first':
            if input_shape[0] not in {1, 3}:
                warnings.warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[0]) + ' input channels.')
            default_shape = (input_shape[0], default_size, default_size)
        else:
            if input_shape[-1] not in {1, 3}:
                warnings.warn(
                    'This model usually expects 1 or 3 input channels. '
                    'However, it was passed an input_shape with ' +
                    str(input_shape[-1]) + ' input channels.')
            default_shape = (default_size, default_size, input_shape[-1])
    else:
        if data_format == 'channels_first':
            default_shape = (3, default_size, default_size)
        else:
            default_shape = (default_size, default_size, 3)
    if weights == 'imagenet' and require_flatten:
        if input_shape is not None:
            if input_shape != default_shape:
                raise ValueError('When setting `include_top=True` '
                                 'and loading `imagenet` weights, '
                                 '`input_shape` should be ' +
                                 str(default_shape) + '.')
        return default_shape
    if input_shape:
        if data_format == 'channels_first':
            if input_shape is not None:
                if len(input_shape) != 3:
                    raise ValueError(
                        '`input_shape` must be a tuple of three integers.')
                if input_shape[0] != 3 and weights == 'imagenet':
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')
                if ((input_shape[1] is not None and input_shape[1] < min_size) or
                   (input_shape[2] is not None and input_shape[2] < min_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_size) + 'x' + str(min_size) +
                                     '; got `input_shape=' +
                                     str(input_shape) + '`')
        else:
            if input_shape is not None:
                if len(input_shape) != 3:
                    raise ValueError(
                        '`input_shape` must be a tuple of three integers.')
                if input_shape[-1] != 3 and weights == 'imagenet':
                    raise ValueError('The input must have 3 channels; got '
                                     '`input_shape=' + str(input_shape) + '`')
                if ((input_shape[0] is not None and input_shape[0] < min_size) or
                   (input_shape[1] is not None and input_shape[1] < min_size)):
                    raise ValueError('Input size must be at least ' +
                                     str(min_size) + 'x' + str(min_size) +
                                     '; got `input_shape=' +
                                     str(input_shape) + '`')
    else:
        if require_flatten:
            input_shape = default_shape
        else:
            if data_format == 'channels_first':
                input_shape = (3, None, None)
            else:
                input_shape = (None, None, 3)
    if require_flatten:
        if None in input_shape:
            raise ValueError('If `include_top` is True, '
                             'you should specify a static `input_shape`. '
                             'Got `input_shape=' + str(input_shape) + '`')
    return input_shape

In [30]:
def _preprocess_input(x, data_format=None, mode='caffe', **kwargs):
    """Preprocesses a tensor or Numpy array encoding a batch of images.
    # Arguments
        x: Input Numpy or symbolic tensor, 3D or 4D.
            The preprocessed data is written over the input data
            if the data types are compatible. To avoid this
            behaviour, `numpy.copy(x)` can be used.
        data_format: Data format of the image tensor/array.
        mode: One of "caffe", "tf" or "torch".
            - caffe: will convert the images from RGB to BGR,
                then will zero-center each color channel with
                respect to the ImageNet dataset,
                without scaling.
            - tf: will scale pixels between -1 and 1,
                sample-wise.
            - torch: will scale pixels between 0 and 1 and then
                will normalize each channel with respect to the
                ImageNet dataset.
    # Returns
        Preprocessed tensor or Numpy array.
    # Raises
        ValueError: In case of unknown `data_format` argument.
    """
    backend, _, _, _ = get_submodules_from_kwargs(kwargs)

    if data_format is None:
        data_format = backend.image_data_format()
    if data_format not in {'channels_first', 'channels_last'}:
        raise ValueError('Unknown data_format ' + str(data_format))

    if isinstance(x, np.ndarray):
        return _preprocess_numpy_input(x, data_format=data_format,
                                       mode=mode, **kwargs)
    else:
        return _preprocess_symbolic_input(x, data_format=data_format,
                                          mode=mode, **kwargs)

In [31]:
IMAGENET_WEIGHTS_PATH = (
    'https://github.com/Callidior/keras-applications/'
    'releases/download/efficientnet/')

IMAGENET_WEIGHTS_HASHES = {
    'efficientnet-b0': ('163292582f1c6eaca8e7dc7b51b01c61'
                        '5b0dbc0039699b4dcd0b975cc21533dc',
                        'c1421ad80a9fc67c2cc4000f666aa507'
                        '89ce39eedb4e06d531b0c593890ccff3'),
    'efficientnet-b1': ('d0a71ddf51ef7a0ca425bab32b7fa7f1'
                        '6043ee598ecee73fc674d9560c8f09b0',
                        '75de265d03ac52fa74f2f510455ba64f'
                        '9c7c5fd96dc923cd4bfefa3d680c4b68'),
    'efficientnet-b2': ('bb5451507a6418a574534aa76a91b106'
                        'f6b605f3b5dde0b21055694319853086',
                        '433b60584fafba1ea3de07443b74cfd3'
                        '2ce004a012020b07ef69e22ba8669333'),
    'efficientnet-b3': ('03f1fba367f070bd2545f081cfa7f3e7'
                        '6f5e1aa3b6f4db700f00552901e75ab9',
                        'c5d42eb6cfae8567b418ad3845cfd63a'
                        'a48b87f1bd5df8658a49375a9f3135c7'),
    'efficientnet-b4': ('98852de93f74d9833c8640474b2c698d'
                        'b45ec60690c75b3bacb1845e907bf94f',
                        '7942c1407ff1feb34113995864970cd4'
                        'd9d91ea64877e8d9c38b6c1e0767c411'),
    'efficientnet-b5': ('30172f1d45f9b8a41352d4219bf930ee'
                        '3339025fd26ab314a817ba8918fefc7d',
                        '9d197bc2bfe29165c10a2af8c2ebc675'
                        '07f5d70456f09e584c71b822941b1952'),
    'efficientnet-b6': ('f5270466747753485a082092ac9939ca'
                        'a546eb3f09edca6d6fff842cad938720',
                        '1d0923bb038f2f8060faaf0a0449db4b'
                        '96549a881747b7c7678724ac79f427ed'),
    'efficientnet-b7': ('876a41319980638fa597acbbf956a82d'
                        '10819531ff2dcb1a52277f10c7aefa1a',
                        '60b56ff3a8daccc8d96edfd40b204c11'
                        '3e51748da657afd58034d54d3cec2bac')
}

NS_WEIGHTS_PATH = 'https://github.com/qubvel/efficientnet/releases/download/v0.0.1/'
NS_WEIGHTS_HASHES = {
    'efficientnet-b0': ('5e376ca93bc6ba60f5245d13d44e4323', 'a5b48ae7547fc990c7e4f3951230290d'),
    'efficientnet-b1': ('79d29151fdaec95ac78e1ca97fc09634', '4d35baa41ca36f175506a33918f7e334'),
    'efficientnet-b2': ('8c643222ffb73a2bfdbdf90f2cde01af', 'e496e531f41242598288ff3a4b4199f9'),
    'efficientnet-b3': ('3b29e32602dad75d1f575d9ded00f930', '47da5b154de1372b557a65795d3e6135'),
    'efficientnet-b4': ('c000bfa03bf3c93557851b4e1fe18f51', '47c10902a4949eec589ab92fe1c35ed8'),
    'efficientnet-b5': ('8a920cd4ee793f53c251a1ecd3a5cee6', '4d53ef3544d4114e2d8080d6d777a74c'),
    'efficientnet-b6': ('cc69df409516ab57e30e51016326853e', '71f96d7e15d9f891f3729b4f4e701f77'),
    'efficientnet-b7': ('1ac825752cbc26901c8952e030ae4dd9', 'e112b00c464fe929b821edbb35d1af55')
}

In [32]:
_KERAS_BACKEND = None
_KERAS_LAYERS = None
_KERAS_MODELS = None
_KERAS_UTILS = None

def get_submodules_from_kwargs(kwargs):
    #backend = kwargs.get('backend', _KERAS_BACKEND)
    backend=keras.backend
    #layers = kwargs.get('layers', _KERAS_LAYERS)
    layers = keras.layers
    #models = kwargs.get('models', _KERAS_MODELS)
    models = tf.keras.models
    utils = kwargs.get('utils', _KERAS_UTILS)
    for key in kwargs.keys():
        if key not in ['backend', 'layers', 'models', 'utils']:
            raise TypeError('Invalid keyword argument: %s', key)
    return backend, layers, models, utils

In [55]:
# Copyright 2019 The TensorFlow Authors, Pavel Yakubovskiy, Björn Barz. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Contains definitions for EfficientNet model.
[1] Mingxing Tan, Quoc V. Le
  EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks.
  ICML'19, https://arxiv.org/abs/1905.11946
"""

# Code of this model implementation is mostly written by
# Björn Barz ([@Callidior](https://github.com/Callidior))

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import math
import string
import collections

from six.moves import xrange
#from keras_applications.imagenet_utils import _obtain_input_shape
#from keras_applications.imagenet_utils import preprocess_input as _preprocess_input

#from . import get_submodules_from_kwargs
#from .weights import IMAGENET_WEIGHTS_PATH, IMAGENET_WEIGHTS_HASHES, NS_WEIGHTS_HASHES, NS_WEIGHTS_PATH

backend = None
layers = None
models = None
keras_utils = None

BlockArgs = collections.namedtuple('BlockArgs', [
    'kernel_size', 'num_repeat', 'input_filters', 'output_filters',
    'expand_ratio', 'id_skip', 'strides', 'se_ratio'
])
# defaults will be a public argument for namedtuple in Python 3.7
# https://docs.python.org/3/library/collections.html#collections.namedtuple
BlockArgs.__new__.__defaults__ = (None,) * len(BlockArgs._fields)

DEFAULT_BLOCKS_ARGS = [
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=32, output_filters=16,
              expand_ratio=1, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=2, input_filters=16, output_filters=24,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=2, input_filters=24, output_filters=40,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=3, input_filters=40, output_filters=80,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=3, input_filters=80, output_filters=112,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25),
    BlockArgs(kernel_size=5, num_repeat=4, input_filters=112, output_filters=192,
              expand_ratio=6, id_skip=True, strides=[2, 2], se_ratio=0.25),
    BlockArgs(kernel_size=3, num_repeat=1, input_filters=192, output_filters=320,
              expand_ratio=6, id_skip=True, strides=[1, 1], se_ratio=0.25)
]

CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        # EfficientNet actually uses an untruncated normal distribution for
        # initializing conv layers, but keras.initializers.VarianceScaling use
        # a truncated distribution.
        # We decided against a custom initializer for better serializability.
        'distribution': 'normal'
    }
}

DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}


def preprocess_input(x, **kwargs):
    kwargs = {k: v for k, v in kwargs.items() if k in ['backend', 'layers', 'models', 'utils']}
    return _preprocess_input(x, mode='torch', **kwargs)


def get_swish(**kwargs):
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    def swish(x):
        """Swish activation function: x * sigmoid(x).
        Reference: [Searching for Activation Functions](https://arxiv.org/abs/1710.05941)
        """

        if backend.backend() == 'tensorflow':
            try:
                # The native TF implementation has a more
                # memory-efficient gradient implementation
                return backend.tf.nn.swish(x)
            except AttributeError:
                pass

        return x * backend.sigmoid(x)

    return swish


def get_dropout(**kwargs):
    """Wrapper over custom dropout. Fix problem of ``None`` shape for tf.keras.
    It is not possible to define FixedDropout class as global object,
    because we do not have modules for inheritance at first time.
    Issue:
        https://github.com/tensorflow/tensorflow/issues/30946
    """
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    class FixedDropout(layers.Dropout):
        def _get_noise_shape(self, inputs):
            if self.noise_shape is None:
                return self.noise_shape

            symbolic_shape = backend.shape(inputs)
            noise_shape = [symbolic_shape[axis] if shape is None else shape
                           for axis, shape in enumerate(self.noise_shape)]
            return tuple(noise_shape)

    #return FixedDropout
    return Dropout


def round_filters(filters, width_coefficient, depth_divisor):
    """Round number of filters based on width multiplier."""

    filters *= width_coefficient
    new_filters = int(filters + depth_divisor / 2) // depth_divisor * depth_divisor
    new_filters = max(depth_divisor, new_filters)
    # Make sure that round down does not go down by more than 10%.
    if new_filters < 0.9 * filters:
        new_filters += depth_divisor
    return int(new_filters)


def round_repeats(repeats, depth_coefficient):
    """Round number of repeats based on depth multiplier."""

    return int(math.ceil(depth_coefficient * repeats))


def mb_conv_block(inputs, block_args, activation, drop_rate=None, prefix='', ):
    """Mobile Inverted Residual Bottleneck."""

    has_se = (block_args.se_ratio is not None) and (0 < block_args.se_ratio <= 1)
    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1

    # workaround over non working dropout with None in noise_shape in tf.keras
    Dropout = get_dropout(
        backend=backend,
        layers=layers,
        models=models,
        utils=keras_utils
    )

    # Expansion phase
    filters = block_args.input_filters * block_args.expand_ratio
    if block_args.expand_ratio != 1:
        x = layers.Conv2D(filters, 1,
                          padding='same',
                          use_bias=False,
                          kernel_initializer=CONV_KERNEL_INITIALIZER,
                          name=prefix + 'expand_conv')(inputs)
        x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'expand_bn')(x)
        x = layers.Activation(activation, name=prefix + 'expand_activation')(x)
    else:
        x = inputs

    # Depthwise Convolution
    x = layers.DepthwiseConv2D(block_args.kernel_size,
                               strides=block_args.strides,
                               padding='same',
                               use_bias=False,
                               depthwise_initializer=CONV_KERNEL_INITIALIZER,
                               name=prefix + 'dwconv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'bn')(x)
    x = layers.Activation(activation, name=prefix + 'activation')(x)

    # Squeeze and Excitation phase
    if has_se:
        num_reduced_filters = max(1, int(
            block_args.input_filters * block_args.se_ratio
        ))
        se_tensor = layers.GlobalAveragePooling2D(name=prefix + 'se_squeeze')(x)

        target_shape = (1, 1, filters) if backend.image_data_format() == 'channels_last' else (filters, 1, 1)
        se_tensor = layers.Reshape(target_shape, name=prefix + 'se_reshape')(se_tensor)
        se_tensor = layers.Conv2D(num_reduced_filters, 1,
                                  activation=activation,
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_reduce')(se_tensor)
        se_tensor = layers.Conv2D(filters, 1,
                                  activation='sigmoid',
                                  padding='same',
                                  use_bias=True,
                                  kernel_initializer=CONV_KERNEL_INITIALIZER,
                                  name=prefix + 'se_expand')(se_tensor)
        if backend.backend() == 'theano':
            # For the Theano backend, we have to explicitly make
            # the excitation weights broadcastable.
            pattern = ([True, True, True, False] if backend.image_data_format() == 'channels_last'
                       else [True, False, True, True])
            se_tensor = layers.Lambda(
                lambda x: backend.pattern_broadcast(x, pattern),
                name=prefix + 'se_broadcast')(se_tensor)
        x = layers.multiply([x, se_tensor], name=prefix + 'se_excite')

    # Output phase
    x = layers.Conv2D(block_args.output_filters, 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name=prefix + 'project_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=prefix + 'project_bn')(x)
    if block_args.id_skip and all(
            s == 1 for s in block_args.strides
    ) and block_args.input_filters == block_args.output_filters:
        if drop_rate and (drop_rate > 0):
            x = Dropout(drop_rate,
                        noise_shape=(None, 1, 1, 1),
                        name=prefix + 'drop')(x)
        x = layers.add([x, inputs], name=prefix + 'add')

    return x


def EfficientNet(width_coefficient,
                 depth_coefficient,
                 default_resolution,
                 dropout_rate=0.2,
                 drop_connect_rate=0.2,
                 depth_divisor=8,
                 blocks_args=DEFAULT_BLOCKS_ARGS,
                 model_name='efficientnet',
                 include_top=True,
                 weights='imagenet',
                 input_tensor=None,
                 input_shape=None,
                 pooling=None,
                 classes=1000,
                 **kwargs):
    """Instantiates the EfficientNet architecture using given scaling coefficients.
    Optionally loads weights pre-trained on ImageNet.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        width_coefficient: float, scaling coefficient for network width.
        depth_coefficient: float, scaling coefficient for network depth.
        default_resolution: int, default input image size.
        dropout_rate: float, dropout rate before final classifier layer.
        drop_connect_rate: float, dropout rate at skip connections.
        depth_divisor: int.
        blocks_args: A list of BlockArgs to construct block modules.
        model_name: string, model name.
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False.
            It should have exactly 3 inputs channels.
        pooling: optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional layer.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional layer, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True, and
            if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    global backend, layers, models, keras_utils
    backend, layers, models, keras_utils = get_submodules_from_kwargs(kwargs)

    if not (weights in {'imagenet', 'noisy-student', None} or os.path.exists(weights)):
        raise ValueError('The `weights` argument should be either '
                         '`None` (random initialization), `imagenet` '
                         '(pre-training on ImageNet), '
                         'or the path to the weights file to be loaded.')

    if weights == 'imagenet' and include_top and classes != 1000:
        raise ValueError('If using `weights` as `"imagenet"` with `include_top`'
                         ' as true, `classes` should be 1000')

    # Determine proper input shape
    input_shape = _obtain_input_shape(input_shape,
                                      default_size=default_resolution,
                                      min_size=32,
                                      data_format=backend.image_data_format(),
                                      require_flatten=include_top,
                                      weights=weights)

    if input_tensor is None:
        img_input = layers.Input(shape=input_shape)
    else:
        if backend.backend() == 'tensorflow':
            from tensorflow.python.keras.backend import is_keras_tensor
        else:
            is_keras_tensor = backend.is_keras_tensor
        if not is_keras_tensor(input_tensor):
            img_input = layers.Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    bn_axis = 3 if backend.image_data_format() == 'channels_last' else 1
    #activation = get_swish(**kwargs)
    activation = "relu"

    # Build stem
    x = img_input
    x = layers.Conv2D(round_filters(32, width_coefficient, depth_divisor), 3,
                      strides=(2, 2),
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name='stem_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='stem_bn')(x)
    x = layers.Activation(activation, name='stem_activation')(x)

    # Build blocks
    num_blocks_total = sum(block_args.num_repeat for block_args in blocks_args)
    block_num = 0
    for idx, block_args in enumerate(blocks_args):
        assert block_args.num_repeat > 0
        # Update block input and output filters based on depth multiplier.
        block_args = block_args._replace(
            input_filters=round_filters(block_args.input_filters,
                                        width_coefficient, depth_divisor),
            output_filters=round_filters(block_args.output_filters,
                                         width_coefficient, depth_divisor),
            num_repeat=round_repeats(block_args.num_repeat, depth_coefficient))

        # The first block needs to take care of stride and filter size increase.
        drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
        x = mb_conv_block(x, block_args,
                          activation=activation,
                          drop_rate=drop_rate,
                          prefix='block{}a_'.format(idx + 1))
        block_num += 1
        if block_args.num_repeat > 1:
            # pylint: disable=protected-access
            block_args = block_args._replace(
                input_filters=block_args.output_filters, strides=[1, 1])
            # pylint: enable=protected-access
            for bidx in xrange(block_args.num_repeat - 1):
                drop_rate = drop_connect_rate * float(block_num) / num_blocks_total
                block_prefix = 'block{}{}_'.format(
                    idx + 1,
                    string.ascii_lowercase[bidx + 1]
                )
                x = mb_conv_block(x, block_args,
                                  activation=activation,
                                  drop_rate=drop_rate,
                                  prefix=block_prefix)
                block_num += 1

    # Build top
    x = layers.Conv2D(round_filters(1280, width_coefficient, depth_divisor), 1,
                      padding='same',
                      use_bias=False,
                      kernel_initializer=CONV_KERNEL_INITIALIZER,
                      name='top_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, name='top_bn')(x)
    x = layers.Activation(activation, name='top_activation')(x)
    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        if dropout_rate and dropout_rate > 0:
            x = layers.Dropout(dropout_rate, name='top_dropout')(x)
        x = layers.Dense(classes,
                         activation='softmax',
                         kernel_initializer=DENSE_KERNEL_INITIALIZER,
                         name='probs')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D(name='max_pool')(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    if input_tensor is not None:
        inputs = keras_utils.get_source_inputs(input_tensor)
    else:
        inputs = img_input

    # Create model.
    model = models.Model(inputs, x, name=model_name)

    # Load weights.
    if weights == 'imagenet':

        if include_top:
            file_name = model_name + '_weights_tf_dim_ordering_tf_kernels_autoaugment.h5'
            file_hash = IMAGENET_WEIGHTS_HASHES[model_name][0]
        else:
            file_name = model_name + '_weights_tf_dim_ordering_tf_kernels_autoaugment_notop.h5'
            file_hash = IMAGENET_WEIGHTS_HASHES[model_name][1]
        weights_path = keras_utils.get_file(
            file_name,
            IMAGENET_WEIGHTS_PATH + file_name,
            cache_subdir='models',
            file_hash=file_hash,
        )
        model.load_weights(weights_path)

    elif weights == 'noisy-student':

        if include_top:
            file_name = "{}_{}.h5".format(model_name, weights)
            file_hash = NS_WEIGHTS_HASHES[model_name][0]
        else:
            file_name = "{}_{}_notop.h5".format(model_name, weights)
            file_hash = NS_WEIGHTS_HASHES[model_name][1]
        weights_path = keras_utils.get_file(
            file_name,
            NS_WEIGHTS_PATH + file_name,
            cache_subdir='models',
            file_hash=file_hash,
        )
        model.load_weights(weights_path)

    elif weights is not None:
        model.load_weights(weights)

    return model


def EfficientNetB0(
        include_top=True,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        **kwargs
):
    return EfficientNet(
        1.0, 1.0, 224, 0.2,
        model_name='efficientnet-b0',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetB1(
        include_top=True,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        **kwargs
):
    return EfficientNet(
        1.0, 1.1, 240, 0.2,
        model_name='efficientnet-b1',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetB2(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet(
        1.1, 1.2, 260, 0.3,
        model_name='efficientnet-b2',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetB3(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   **kwargs):
    return EfficientNet(
        1.2, 1.4, 300, 0.3,
        model_name='efficientnet-b3',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetB4(
        include_top=True,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        **kwargs
):
    return EfficientNet(
        1.4, 1.8, 380, 0.4,
        model_name='efficientnet-b4',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetB5(
        include_top=True,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        **kwargs
):
    return EfficientNet(
        1.6, 2.2, 456, 0.4,
        model_name='efficientnet-b5',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetB6(
        include_top=True,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        **kwargs
):
    return EfficientNet(
        1.8, 2.6, 528, 0.5,
        model_name='efficientnet-b6',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetB7(
        include_top=True,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        **kwargs
):
    return EfficientNet(
        2.0, 3.1, 600, 0.5,
        model_name='efficientnet-b7',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )


def EfficientNetL2(
        include_top=True,
        weights='imagenet',
        input_tensor=None,
        input_shape=None,
        pooling=None,
        classes=1000,
        **kwargs
):
    return EfficientNet(
        4.3, 5.3, 800, 0.5,
        model_name='efficientnet-l2',
        include_top=include_top, weights=weights,
        input_tensor=input_tensor, input_shape=input_shape,
        pooling=pooling, classes=classes,
        **kwargs
    )

setattr(EfficientNetB0, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetB1, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetB2, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetB3, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetB4, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetB5, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetB6, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetB7, '__doc__', EfficientNet.__doc__)
setattr(EfficientNetL2, '__doc__', EfficientNet.__doc__)

## Configurando EfficientNet para obtener capa tipo modelo

In [56]:
def inject_tfkeras_modules(func):
    import tensorflow.keras as tfkeras
    #@functools.wraps(func)
    def wrapper(*args, **kwargs):
        kwargs['backend'] = tfkeras.backend
        kwargs['layers'] = tfkeras.layers
        kwargs['models'] = tfkeras.models
        kwargs['utils'] = tfkeras.utils
        return func(*args, **kwargs)

    return wrapper

In [57]:
EfficientNetB0 = inject_tfkeras_modules(EfficientNetB0)
EfficientNetB1 = inject_tfkeras_modules(EfficientNetB1)
EfficientNetB2 = inject_tfkeras_modules(EfficientNetB2)
EfficientNetB3 = inject_tfkeras_modules(EfficientNetB3)
EfficientNetB4 = inject_tfkeras_modules(EfficientNetB4)
EfficientNetB5 = inject_tfkeras_modules(EfficientNetB5)
EfficientNetB6 = inject_tfkeras_modules(EfficientNetB6)
EfficientNetB7 = inject_tfkeras_modules(EfficientNetB7)
EfficientNetL2 = inject_tfkeras_modules(EfficientNetL2)

## Arquitectura del modelo

In [58]:
dropout = 0.2

In [59]:
# Creando el modelo EfficientNet con pesos preentrenados en ImageNet
#rnet = efn.EfficientNetB7(weights=None, include_top=False, input_shape=[128,128,3])
#rnet = tf.keras.applications.EfficientNetB7(weights=None, include_top=False, input_shape=[128,128,3]) # Tensoorflow 2.3.0 en adelante
rnet = EfficientNetB7(weights=None, include_top=False, input_shape=[128,128,3])
rnet = tf.keras.Sequential(rnet)
rnet.add(Dropout(rate=dropout))
rnet.add(Flatten())
rnet.add(Dense(4096, activation='relu'))
rnet.add(Dropout(rate=dropout))
rnet.add(Dense(4096, activation='relu'))
rnet.add(Dropout(rate=dropout))
rnet.add(Dense(1, activation='sigmoid'))

In [60]:
rnet.layers

In [61]:
rnet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Model)      (None, 4, 4, 2560)        64097680  
_________________________________________________________________
dropout_3 (Dropout)          (None, 4, 4, 2560)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 40960)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 4096)              167776256 
_________________________________________________________________
dropout_4 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_5 (Dropout)          (None, 4096)             

In [62]:
type(rnet)

tensorflow.python.keras.engine.sequential.Sequential

In [63]:
rnet.load_weights('EFNB7_weights.h5')

In [64]:
rnet.save('modelo.h5')

In [65]:
!ls

EFNB7_weights.h5  modelo.h5  modelo.tar.gz  T0032.1.1.D.2016-10-29.16.npy


## Comprimiendo modelo EfiicientNet en formato tar.gz

In [66]:
!tar -zcvf modelo.tar.gz modelo.h5

modelo.h5


In [67]:
!ls

EFNB7_weights.h5  modelo.h5  modelo.tar.gz  T0032.1.1.D.2016-10-29.16.npy


In [68]:
# Guardando la dirección del modelo tar.gz
model_path = '/home/wsuser/work/modelo.tar.gz'

In [69]:
rnet.predict(X_prueba_128)

array([[0.02154853]], dtype=float32)

## Modelo de Tensorflow desplegado en la nube

Aquí se describen los pasos para almaenar el modelo EfficientNet en el repositorio de Watson Machine Learningn utilizando el SDK de IBM Watson Machine Learning.

In [70]:
tf.__version__

'2.1.0'

### Publicando modelo

Definiendo nombre del modelo, nombre del autor y el software específico.

In [71]:
sofware_spec_uid = client.software_specifications.get_id_by_name("tensorflow_2.1-py3.7")

In [72]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'EfficientNetB7',
            client.repository.ModelMetaNames.TYPE: 'tensorflow_2.1',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=model_path,
    meta_props=metadata)

### Obteniendo detalles del modelo

In [73]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "software_spec": {
      "id": "cXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
      "name": "tensorflow_2.1-py3.7"
    },
    "type": "tensorflow_2.1"
  },
  "metadata": {
    "created_at": "2020-12-04T02:17:10.369Z",
    "id": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX",
    "modified_at": "2020-12-04T02:17:37.235Z",
    "name": "EfficientNetB7",
    "owner": "IBMid-550009WDCT",
    "space_id": "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
  },
  "system": {
    "warnings": []
  }
}


In [74]:
models_details = client.repository.list_models()

------------------------------------  --------------  ------------------------  --------------
ID                                    NAME            CREATED                   TYPE
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7  2020-12-04T02:17:10.002Z  tensorflow_2.1
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7  2020-12-04T02:07:29.002Z  tensorflow_2.1
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7  2020-12-04T00:56:48.002Z  tensorflow_2.1
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7  2020-12-04T00:23:59.002Z  tensorflow_2.1
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7  2020-12-03T23:26:27.002Z  tensorflow_2.1
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  EfficientNetB7  2020-12-03T22:41:42.002Z  tensorflow_2.1
------------------------------------  --------------  ------------------------  --------------


## Desplegando y almacenando en una nube

Se almacena el modelo para poder obtener un score de forma online uutilizando solicitudes tipo con arreglos tipo json con ayuda de IBM Watson Machine Learning SDK.

### Creando modelo desplegado

#### Creando despliegue online para el modelo publicado.

In [75]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of external Tensorflow model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX' started

#######################################################################################


initializing................
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
------------------------------------------------------------------------------------------------




In [76]:
rnet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b7 (Model)      (None, 4, 4, 2560)        64097680  
_________________________________________________________________
dropout_3 (Dropout)          (None, 4, 4, 2560)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 40960)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 4096)              167776256 
_________________________________________________________________
dropout_4 (Dropout)          (None, 4096)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 4096)              16781312  
_________________________________________________________________
dropout_5 (Dropout)          (None, 4096)             

In [77]:
deployment_uid = client.deployments.get_uid(created_deployment)

Ahora podemos imprimir el "online scoring endpoint"

In [78]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions


You can also list existing deployments.

In [79]:
client.deployments.list()

------------------------------------  ---------------------------------------  ------  ------------------------
GUID                                  NAME                                     STATE   CREATED
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  ready   2020-12-04T02:17:40.026Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  failed  2020-12-04T02:08:15.442Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  failed  2020-12-04T00:59:56.165Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  failed  2020-12-04T00:24:36.800Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  failed  2020-12-03T23:28:05.407Z
XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX  Deployment of external Tensorflow model  failed  2020-12-03T22:43:00.904Z
------------------------------------  ---------------------------------------  ------  ------------------------


### Obteniendo detalles del "despliegue"

In [80]:
client.deployments.get_details(deployment_uid)

{'entity': {'asset': {'id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'},
  'custom': {},
  'deployed_asset_type': 'model',
  'hardware_spec': {'id': 'Not_Applicable', 'name': 'XS', 'num_nodes': 1},
  'name': 'Deployment of external Tensorflow model',
  'online': {},
  'space_id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
  'status': {'online_url': {'url': 'https://us-south.ml.cloud.ibm.com/ml/v4/deployments/XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX/predictions'},
   'state': 'ready'}},
 'metadata': {'created_at': '2020-12-04T02:17:40.026Z',
  'id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX',
  'modified_at': '2020-12-04T02:17:40.026Z',
  'name': 'Deployment of external Tensorflow model',
  'owner': 'IBMid-550009WDCT',
  'space_id': 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'}}

### Score

Ahora vamos a realizar una prueba de acceso.

In [95]:
score_0 = X_prueba_128.tolist()

In [98]:
scoring_payload = {"input_data": [{"values": [score_0[0]]}]}

Usando el método ``client.deployments.score()``para probar el modelo en la nube.

In [99]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

Let's print the result of predictions.

In [100]:
import json
print(json.dumps(predictions, indent=2))

{
  "predictions": [
    {
      "id": "dense_5",
      "fields": [
        "prediction",
        "prediction_classes",
        "probability"
      ],
      "values": [
        [
          [
            0.021548686549067497
          ],
          [
            0
          ],
          [
            0.021548686549067497
          ]
        ]
      ]
    }
  ]
}


Imprimimos la probabilidad de cáncer.

In [101]:
# Modelo Online
predictions['predictions'][0]['values'][0][0][0]

0.021548686549067497

In [ ]:
# Modelo "local"
rnet.predict(score_0)[0][0]